
GTFS Wiki: http://wiki.corp.nextbus.com/nextbuswiki/GtfsChecks (need to be on NexBus VPN) 

GTFS Goolge Guideline: https://developers.google.com/transit/gtfs/reference/

# <span style="color:red">Import the needed packages</span>

In [1]:
import pandas as pd
import numpy as np
#import pixiedust as pix # will look into the capabilites of this library

### <span style="color:green">Add the location of your GTFS data</span>

In [2]:
# for windows
# filepath = r"\\cts-con-fld-001\users$\207329\My Documents\qa\vacaville\vacavillecitycoach-ca-us_20181015"

# for linux
filepath = r"/mnt/shared/qa/foothill"

# filepath2 = r"/mnt/shared/qa/seattle-sc/Seattle-sc-ALL_20181126_google_daily_transit"



'''Some functions to make life eaiser
Note if you are using Windows, you'll need to modify the "/" '''




# load file and show top 5 records
def load(fName): 
    f = filepath + "/" + "{}.txt".format(fName)
    global df
    df = pd.read_csv(f)
    return df.info()

# load second file for comparision
def load2(fName): 
    f = filepath2 + "/" + "{}.txt".format(fName)
    global df2
    df2 = pd.read_csv(f)
    return df2.info()

import csv
# short save file
def save(dfName,fName):
    df = dfName
    df.to_csv(filepath + '//' + "{}".format(fName), sep =',', index=False)#, quoting=csv.QUOTE_NONNUMERIC)
    return print("Saved {} to {}".format(fName,filepath))

# short find null
def null(fName):
    return df[(fName).isnull()]

# drop field (field name, file name)
def drop(fName,fileName):
    df.drop(fName, 1, inplace=True)
    df.to_csv(filepath + '//' + '{}_FIXED.txt'.format(fileName), sep =',', index=False)
    return print("Dropped {} field.\n\
Saved '{}_FIXED.txt' to file folder.".format(fName,fileName))

# drop dups in blocks.txt for CNB 
def dups():
    #df[df.block_id.duplicated()]
    df1 = df.drop_duplicates(subset = 'block_id', keep='last').reset_index(drop=True)
    return df1.info()
    
# sort sequence for stop_times.txt file
def sort():
    df.sort_values(['trip_id', 'stop_sequence'])
    return df.head(15)

# shape less than certain amount of points for shapes
def lessthan(x):
    df1 = df.groupby(['shape_id']).count()
    return df1[(df1.shape_pt_lat < x)]

# function to move copy stop_code to stop_id in the stops.txt file
def copyField(newField,origField,fileName):
    df[newField] = df[origField]
    df.to_csv(filepath + '//' + '{}_FIXED.txt'.format(fileName), sep =',', index=False)
    return print("Copied the '{}' field to the '{}' field.\n\
Saved '{}_FIXED.txt' to file folder.".format(origField,newField,fileName))




### work for CNB shape issues
# def shapeFix(indx1,indx2,shapeName):
#     x1 = int(indx1)
#     x2 = int(indx2)
#     global df
#     df = df.drop(df.index[x1:x2]) # drop double shape pnts 
#     df = df[(df.shape_id == '{}'.format(shapeName))] # create new df of just shape pnts
#     df['shape_pt_sequence'] = range(1,1 + len(df)) # re-sequence shape pnts
#     df_cut = df # give unique name to subset of shape pnts

#     # load orig data, drop all of courthou1, append 
#     load('shapes') # load orig df
#     df = df[df.shape_id != '{}'.format(shapeName)] # drop all cases of shape 
#     global df_all
#     df_all = df.append(df_cut,ignore_index=True) # create new df and append df_cut
#     return  print('use df_all')



## Random Work-flows

In [ ]:
## fill na with number, query and output file  
df['block_id'].fillna(999999, inplace=True)
df1 = df[(df.block_id == 999999)]
df['block_id'].unique()
save(df1,'trips_missingBlocks.txt')

# compare two files
# set the field to a list
oldTrips = df2['trip_id'].unique().tolist()
newTrips = df['trip_id'].unique().tolist()

oldBlocks = df2['block_id'].unique().tolist()
newBlocks = df['block_id'].unique().tolist()

# use set and difference to compare the two list
set(oldTrips).difference(newTrips)

set(newBlocks).difference(oldBlocks)

# can alsow use this logic... but the above works just fine
if (40803723 in newTrips):
    print ("Exist")
else:
    print("Nope")

##### seattle-sc 'like' function 
df=df.loc[df['route_id'].isin([100340,102638])]
df
save(df,'trips')
df['service_id'].unique()
df2=df2.loc[df2['route_id'].isin([100340,102638])]

#### create new column based on conditions of other columns
df['timeCheck'] = np.where(df['arrival_time']==df['departure_time'],'yes','no')
df1 = df[(df.timeCheck=='no')]
df['timeCheck'].unique()

# syntax for leading zeros (4 for example)
df['stop_id']= df['stop_id'].apply(lambda x: '{0:0>4}'.format(x))


# <span style="color:red">agency.txt</span>

In [3]:
load('agency')
df.head(25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
agency_name        1 non-null object
agency_url         1 non-null object
agency_timezone    1 non-null object
agency_lang        0 non-null float64
agency_phone       1 non-null object
dtypes: float64(1), object(4)
memory usage: 120.0+ bytes


,agency_name,agency_url,agency_timezone,agency_lang,agency_phone
0,Foothill Transit,http://www.foothilltransit.org,America/Los_Angeles,NaN,800-RIDE-INFO


# <span style="color:red">routes.txt</span>

### Required Fields: 
1. route_id- These are the routes that are shown to the public. Route ID's cannot have leading zeros, asterisks, underscore, dashes and perhaps other characters. 
2. route_type- numberic value with specific meanings:
    * 0 Tram, Streetcar, Light rail
    * 1 Subway, Metro
    * 2 Rail
    * 3 Bus
    * 4 Ferry
    * 5 Cable Car

In [4]:
# just use the funtion 'load()' to load the .txt file and see the first 5 lines ex: loadfile('routes')
load('routes')
df.head(100)
#copyField('route_short_name','route_id','routes')
# dfEOS = df[(df.agency_id == 'EOS')]
# save(dfEOS,'routes')

#### seattle-sc 'like' function
#df = df.loc[df['agency_id']=='EOS']
# df1['route_long_name'] = df1['route_short_name']
# df = df1
# save('routes_NEW.txt')
#df[(df.route_id==100249)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 8 columns):
route_id            39 non-null object
route_short_name    39 non-null object
route_long_name     39 non-null object
route_desc          0 non-null float64
route_type          39 non-null int64
route_url           39 non-null object
route_color         2 non-null object
route_text_color    0 non-null float64
dtypes: float64(2), int64(1), object(5)
memory usage: 2.5+ KB


,route_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,178-152,178,Puente Hills Mall - El Monte Station,NaN,2,http://foothilltransit.org/line/178/,NaN,NaN
1,185-152,185,Azusa - West Covina - Hacienda Heights,NaN,2,http://foothilltransit.org/line/185/,NaN,NaN
2,187-152,187,Pasadena - Azusa,NaN,2,http://foothilltransit.org/line/187/,NaN,NaN
3,188-152,188,Azusa - Montclair,NaN,2,http://foothilltransit.org/line/188/,NaN,NaN
4,190-152,190,El Monte Station - Cal Poly via Ramona,NaN,3,http://foothilltransit.org/line/190/,NaN,NaN
5,194-152,194,El Monte Station - Cal Poly via Valley,NaN,3,http://foothilltransit.org/line/194/,NaN,NaN
6,195-150,195,Pomona Transcenter- Phillips Ranch- Cal Poly,NaN,2,http://foothilltransit.org/line/195/,NaN,NaN
7,197-150,197,Montclair Transcenter - Pomona Transcenter,NaN,2,http://foothilltransit.org/line/197/,NaN,NaN
8,269-152,269,Montebello Town Center - El Monte Station,NaN,2,http://foothilltransit.org/line/269/,NaN,NaN
9,270-152,270,Monrovia - El Monte Station,NaN,3,http://foothilltransit.org/line/270/,NaN,NaN


# <span style="color:red">trips.txt</span>

### Required Fields:
1. route_id- Check to see if there are trips for every route. 
2. service_id- Check to see if the value of this field comes from the calendar.txt file. (below)
3. trip_id
4. direction_id- Can either be 1 or 0. Can omit if all routes are in one direction. 
5. block_id- GTFS list this as optional, NextBus requires it. Make sure there are no missig values. 
6. shape_id- GTFS list this as optional, NextBus requires it. This field links with the shapes.txt file.(below)
   


In [26]:
load('trips')
df.block_id.unique().tolist()
# dffoo = df[(df.block_id == 5254117)]
# dffoo.head()

## dups work
#df[df.block_id.duplicated()]
#df1 = df.drop_duplicates(subset = 'block_id', keep='last').reset_index(drop=True)
#102638

# df1 = df[(df.route_id == '860-150') | (df.route_id == '861-150')]
# df1 = df[(df.trip_headsign == 'Not In Service')]
# df1
# save(df1,'foothill_trips_notInService.txt')




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4579 entries, 0 to 4578
Data columns (total 7 columns):
route_id         4579 non-null object
service_id       4579 non-null object
trip_id          4579 non-null object
trip_headsign    4579 non-null object
direction_id     4579 non-null int64
block_id         4579 non-null int64
shape_id         4579 non-null int64
dtypes: int64(3), object(4)
memory usage: 250.5+ KB


[817516,
 817584,
 817540,
 817781,
 817517,
 817515,
 817557,
 817549,
 817592,
 817533,
 817668,
 817548,
 817602,
 817522,
 817519,
 817525,
 817532,
 817598,
 817576,
 817634,
 817696,
 817687,
 817545,
 817684,
 817560,
 817766,
 817776,
 817529,
 817626,
 817520,
 817783,
 817685,
 817485,
 817581,
 817625,
 817662,
 817779,
 817765,
 817482,
 817541,
 817681,
 817531,
 817606,
 817623,
 817772,
 817534,
 817556,
 817760,
 817650,
 817622,
 817679,
 817619,
 817586,
 817659,
 817587,
 817543,
 817651,
 817618,
 817688,
 817674,
 817672,
 817572,
 817537,
 817597,
 817629,
 817640,
 817694,
 817631,
 817657,
 817514,
 817538,
 817671,
 817670,
 817658,
 817559,
 817680,
 817501,
 817906,
 817931,
 817481,
 817511,
 817908,
 817932,
 817699,
 817648,
 817579,
 817624,
 817933,
 817892,
 817605,
 817612,
 817583,
 817570,
 817512,
 817571,
 817585,
 817550,
 817635,
 817874,
 817582,
 817880,
 817860,
 817655,
 817759,
 817601,
 817836,
 817868,
 817758,
 817678,
 817878,
 817691,
 

# <span style="color:red">stops.txt</span>

### Required Fields:
1. stop_id 
2. stop_lat- Make sure none are zero, or empty.
3. stop_lon- Make sure none are zero, or empty.
    - make sure lat and long are at least five decimal places (most do six) 

In [8]:
load('stops')
df.head(15)

# copyField('stop_code','stop_id','stops')

#### seattle-sc 'like' function
#df_stop = df.loc[df['stop_code'].isin(stopList)]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1964 entries, 0 to 1963
Data columns (total 10 columns):
stop_id           1964 non-null object
stop_code         0 non-null float64
stop_name         1964 non-null object
stop_desc         0 non-null float64
stop_lat          1964 non-null float64
stop_lon          1964 non-null float64
zone_id           0 non-null float64
stop_url          0 non-null float64
location_type     1964 non-null int64
parent_station    8 non-null object
dtypes: float64(6), int64(1), object(3)
memory usage: 153.5+ KB


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,00001,NaN,Temple Ave & Diamond Bar Blvd/ Mission Blvd E,NaN,34.040203,-117.798027,NaN,NaN,0,NaN
1,00002,NaN,Hacienda Blvd and Temple Ave S,NaN,34.030259,-117.954117,NaN,NaN,0,NaN
2,00003,NaN,Hacienda Blvd and Temple Ave N,NaN,34.030307,-117.953758,NaN,NaN,0,NaN
3,00006,NaN,Hacienda Blvd and Nelson Ave S,NaN,34.026069,-117.957811,NaN,NaN,0,NaN
4,00008,NaN,Hacienda Blvd and Nelson Ave N,NaN,34.026215,-117.957569,NaN,NaN,0,NaN
5,00010,NaN,Orange Grove Ave and Glen Ave N,NaN,34.064132,-117.781031,NaN,NaN,0,NaN
6,00012,NaN,Diamond Bar Blvd and Temple Ave N,NaN,34.040352,-117.798222,NaN,NaN,0,NaN
7,00014,NaN,Diamond Bar Blvd and Mountain Laurel Way N,NaN,33.992106,-117.816991,NaN,NaN,0,NaN
8,00015,NaN,Green St and Raymond Ave E,NaN,34.144446,-118.148989,NaN,NaN,0,NaN
9,00016,NaN,Brea Canyon Rd and Lycoming St N,NaN,34.003135,-117.846353,NaN,NaN,0,NaN


# <span style="color:red">stop_times.txt</span>

### Required Fields:
1. trip_id
2. arrival_time- Should be blank for non-scheduled stops
3. depature_times- Should be blank for non-scheduled stops
4. stop_id
5. stop_sequence- Some agencies will sort this field incorrectly, putting all the "1s" first.
    - note* watch out for the timepoints field, if there is a 1, then it is an acutal stop. If there are not enough of 1s, this may cause problems. It's okay to delete this field. 

In [34]:
load('stop_times') 
time = df.departure_time < '02:00'
df[time]
#drop('timepoint','stop_times')
# df.head(100)
# syntax for leading zeros (4 for example)
# df['stop_id']= df['stop_id'].apply(lambda x: '{0:0>4}'.format(x))
# df.head(100)
# df['timepoint'].unique()

# df[(df.arrival_time!=null) & (df.timepoint==1)]
# df[(df.trip_id == 't_31175_b_718_tn_0')]

# drop('timepoint','stop_times')




#### seattle-sc 'like' function
# df_trip = df.loc[df['trip_id'].isin(tripsList)]
# df_trip.sort_values(['trip_id', 'stop_sequence'])
# stopList = df_trip['stop_id'].unique().tolist()
# save(df_trip,'stop_times_NEW.txt


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187835 entries, 0 to 187834
Data columns (total 9 columns):
trip_id                187835 non-null object
arrival_time           187835 non-null object
departure_time         187835 non-null object
stop_id                187835 non-null int64
stop_sequence          187835 non-null int64
pickup_type            187835 non-null int64
drop_off_type          187835 non-null int64
shape_dist_traveled    187835 non-null float64
timepoint              187835 non-null int64
dtypes: float64(1), int64(5), object(3)
memory usage: 12.9+ MB


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled,timepoint
49282,4804394-1901-FT1905-Weekday-82,01:00:00,01:00:00,2028,1,0,0,0.000,1
49283,4804394-1901-FT1905-Weekday-82,01:13:00,01:13:00,2158,2,0,0,5.570,1
49284,4804394-1901-FT1905-Weekday-82,01:30:37,01:30:37,2647,3,0,0,17.117,0
49285,4804394-1901-FT1905-Weekday-82,01:31:00,01:31:00,2924,4,0,0,17.371,1
49286,4804394-1901-FT1905-Weekday-82,01:32:14,01:32:14,2925,5,0,0,17.989,0
49287,4804394-1901-FT1905-Weekday-82,01:45:00,01:45:00,1171,6,0,0,24.409,1
49288,4804394-1901-FT1905-Weekday-82,01:53:00,01:53:00,897,7,0,0,31.653,1
49289,4804394-1901-FT1905-Weekday-82,01:57:00,01:57:00,2594,8,0,0,34.254,1
49449,4804403-1901-FT1905-Weekday-82,01:10:00,01:10:00,3623,1,0,0,0.000,1
49450,4804403-1901-FT1905-Weekday-82,01:12:12,01:12:12,47,2,0,0,0.335,0


# <span style="color:red">shapes.txt</span>

### Required Fields:
1. shape_id
2. shape_pnt_lat- at least five decimal places
3. shape_pnt_lon- at least five deciaml places

In [12]:
load('shapes')
# df.head(50)
df.describe()

# lessthan(100)


# df['shape_pt_sequence'] = df['shape_pt_sequence'].astype(np.int64)
# save(df,'shapesINT.txt')
# df1 = df[(df.shape_id=='J_trip_inbound_pm')]
# save(df1,'J_trip_inbound_pm.csv')

# flip the point sequence 
# df2 = df[(df.shape_id == 'J_trip_inbound_am')]
# drop = df2[::-1].drop(columns=['shape_pt_sequence'])
# drop.loc[:,'shape_pt_sequence'] = range(1, 1 + len(drop))
# # drop

# df3 = df[df.shape_id != 'J_trip_inbound_am']
# df4 = df3.append(drop)
# new = df4[(df4.shape_id == 'J_trip_inbound_am')]
# df4
# save(df4,'shapes_NEW.txt')
# dfJ = df[(df['shape_id'].str.contains('J'))]
# dfJ['shape_id'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76054 entries, 0 to 76053
Data columns (total 5 columns):
shape_id               76054 non-null int64
shape_pt_lat           76054 non-null float64
shape_pt_lon           76054 non-null float64
shape_pt_sequence      76054 non-null int64
shape_dist_traveled    76054 non-null float64
dtypes: float64(3), int64(2)
memory usage: 2.9 MB


,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
count,7.605400e+04,76054.000000,76054.000000,76054.000000,76054.000000
mean,4.274056e+06,34.057977,-117.927061,171428.483551,10.501442
std,2.027416e+06,0.040381,0.130085,146777.324267,8.455084
min,1.780108e+06,33.914790,-118.268027,10001.000000,0.000000
25%,2.700074e+06,34.030039,-118.001340,50057.000000,4.019000
50%,4.820110e+06,34.062579,-117.929118,120589.000000,8.306500
75%,4.980118e+06,34.079085,-117.829664,250015.000000,14.885000
max,8.610010e+06,34.151759,-117.689463,710012.000000,41.585000


# <span style="color:red">calendar.txt</span>

### Required:
Run the calendar script if more than one service record is defined for the day of the week.
* If there is more than one service defined for the week, then run the calendar.py script.


In [14]:
load('calendar')
df.head(35)


#### seattle-sc 'like' function
# df_cal = df.loc[df['service_id'].isin(serviceList)]
# df_cal

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 10 columns):
service_id    8 non-null object
monday        8 non-null int64
tuesday       8 non-null int64
wednesday     8 non-null int64
thursday      8 non-null int64
friday        8 non-null int64
saturday      8 non-null int64
sunday        8 non-null int64
start_date    8 non-null int64
end_date      8 non-null int64
dtypes: int64(9), object(1)
memory usage: 720.0+ bytes


,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,1901-FT1905-Sunday-82,0,0,0,0,0,0,1,20190616,20190922
1,1901-FT1905-Weekday-82,1,1,1,1,1,0,0,20190617,20190927
2,1901-FT1905-Saturday-82,0,0,0,0,0,1,0,20190622,20190928
3,1901-Pomona-Sunday-62,0,0,0,0,0,0,1,20190616,20190922
4,1901-Pomona-Weekday-62,1,1,1,1,1,0,0,20190617,20190927
5,1901-Pomona-Weekday-62-1111000,1,1,1,1,0,0,0,20190617,20190927
6,1901-Pomona-Weekday-62-0000100,0,0,0,0,1,0,0,20190617,20190927
7,1901-Pomona-Saturday-62,0,0,0,0,0,1,0,20190622,20190928


# <span style="color:red">calendar_dates.txt</span>

### Required:
This is an optional file, but is used to mark special holidays. Make sure the dates make sense.

In [16]:
load('calendar_dates')
df.head(50)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
service_id        12 non-null object
date              12 non-null int64
exception_type    12 non-null int64
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes


,service_id,date,exception_type
0,1901-FT1905-Sunday-82,20190704,1
1,1901-FT1905-Sunday-82,20190902,1
2,1901-FT1905-Weekday-82,20190704,2
3,1901-FT1905-Weekday-82,20190902,2
4,1901-Pomona-Sunday-62,20190704,1
5,1901-Pomona-Sunday-62,20190902,1
6,1901-Pomona-Weekday-62,20190704,2
7,1901-Pomona-Weekday-62-1111000,20190704,2
8,1901-Pomona-Weekday-62-0000100,20190704,2
9,1901-Pomona-Weekday-62,20190902,2
